# 콘텐츠, 인기도 기반 책 제공 서비스

In [1]:
import json
import pandas as pd
import numpy as np

import requests

from tqdm import tqdm_notebook

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
# import SVD
# import accuracy

import warnings; warnings.simplefilter('ignore')

In [3]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

# A. 성인들이 읽는 책을 책소개를 통해 비슷한 책 추천해주기

## 1) 도서관 데이터의 책 소개 크롤링으로 가져오기

- 교보문고 사이트를 활용

In [16]:
# 데이터 불러오기
adult_final_df=pd.read_excel('./data/final/content_re/adult_final_df.xlsx')

In [17]:
len(adult_result)

995

In [9]:
# 크롤링
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
from urllib.error import URLError

In [10]:
# 오픈을 할때 예외처리를 통해 만약 오류가 생기면 원인을 알수 있게 함수 있음
try :
    html = urlopen('https://pythondojang.bitbucket.io/weather/observation/currentweather.html')
except HTTPError as he :
    print('http error')
except URLError as ue :
    print('url error')
else :
    soup = BeautifulSoup(html.read(), 'html.parser')

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
import time

In [ ]:
# isbn 리스트 생성
adult_isbn_list = list(adult_result['isbn13'].astype(str))

In [ ]:
# 자동화 웹 페이지 열기
path = './driver/chromedriver.exe'
driver = webdriver.Chrome(path)
driver

In [ ]:
isbn_url = 'http://www.kyobobook.co.kr/product/detailViewKor.laf?ejkGb=KOR&mallGb=KOR&barcode='
rest_url = '&orderClick=LAG&Kc='

In [ ]:
intro = []

for i in tqdm_notebook(range(len(adult_isbn_list))) :
    url = isbn_url + str(adult_isbn_list[i]) + rest_url
    driver.get(url)
    
    a_list = driver.find_elements_by_css_selector('div.box_detail_article')
    time.sleep(1.5)
    a_list = driver.find_elements_by_css_selector('div.box_detail_article')
    
    if len(a_list) > 0 :
        intro.append(a_list[0].text)
    else :
        intro.append('')
        continue

In [12]:
adult_final_df=adult_result

In [ ]:
adult_final_df['describe']=intro

In [18]:
adult_final_df.head(3)

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,loan_count,bookImageURL,keyword,describe
0,116,116,존리의 부자되기 습관 :대한민국 경제독립 액션 플랜,존리 지음,지식노마드,2020.0,9791187481720,13320.0,NaN,327.04,1371,https://image.aladin.co.kr/product/22631/95/co...,"['펀드', '경제독립', '투자', '금융문맹', '부자', '시작', '한국',...",돈을 위해 일하지 말고 돈이 당신을 위해 일하게 하라!!\n열심히 일하는데 왜 돈에...
1,469,469,당신이 옳다 :정혜신의 적정심리학,정혜신,NaN,NaN,9788965746669,NaN,NaN,189.00,684,http://image.kyobobook.co.kr/images/book/large...,"['공감', '심리', '치유', '심리치유', '콤플렉스', '내공']",안정적인 일상을 위해 스스로 마음의 문제를 해결할 수 있도록 도와주는 공감 행동지침...
2,648,648,남아 있는 시간을 위하여:100세 철학자의 대표산문선,김형석,NaN,NaN,9788934980629,NaN,NaN,104.00,565,http://image.kyobobook.co.kr/images/book/large...,NaN,인생의 마지막에 가까워질수록 더욱 간절해지는 선하고 아름다운 삶을 향한 고민의 소산...


In [19]:
adult_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                888 non-null    int64  
 1   ranking           888 non-null    int64  
 2   bookname          888 non-null    object 
 3   authors           888 non-null    object 
 4   publisher         883 non-null    object 
 5   publication_year  875 non-null    float64
 6   isbn13            888 non-null    int64  
 7   addition_symbol   872 non-null    float64
 8   vol               84 non-null     float64
 9   class_no          876 non-null    float64
 10  loan_count        888 non-null    int64  
 11  bookImageURL      888 non-null    object 
 12  keyword           842 non-null    object 
 13  describe          888 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 97.2+ KB


In [20]:
# 책소개가 없는 데이터지우기
adult_final_df=adult_final_df[pd.notnull(adult_final_df['describe'])]

In [21]:
# 인덱스 정리
adult_final_df.reset_index(drop=True,inplace=True)

In [22]:
adult_final_df['bookname'][4]

'인생 수업'

## 2) 책 소개를 형용사 + 명사 데이터로 바꾸기

In [25]:
book_intro=[]
for i in range(len(adult_final_df)):
    sentences_tag = []
    lines=adult_final_df['describe'][i].replace('\n',' ').split('.')
    twitter = Twitter()
    for sentence in lines:
        morph = twitter.pos(sentence)
        sentences_tag.append(morph)
    noun_adj_list = []
    for sentence1 in sentences_tag:
        for word, tag in sentence1:
            if tag in ['Noun','Adjective']:
                noun_adj_list.append(word)
    book_intro.append(noun_adj_list)

In [26]:
len(book_intro)

888

In [27]:
adult_final_df['book_intro']=book_intro

In [28]:
intro=[]
for i in range(len(adult_final_df)):
    s=' '.join(adult_final_df['book_intro'][i])
    intro.append(s)

In [29]:
adult_final_df['intro']=intro

In [30]:
adult_final_df['describe'].head(3)

0    돈을 위해 일하지 말고 돈이 당신을 위해 일하게 하라!!\n열심히 일하는데 왜 돈에...
1    안정적인 일상을 위해 스스로 마음의 문제를 해결할 수 있도록 도와주는 공감 행동지침...
2    인생의 마지막에 가까워질수록 더욱 간절해지는 선하고 아름다운 삶을 향한 고민의 소산...
Name: describe, dtype: object

In [31]:
adult_final_df['intro'].head(3)

0    돈 위해 일 돈 당신 위해 일 하라 일 왜 돈 삶 살 있는가 한국인 세계 나라 머리...
1    안정 일상 위해 스스로 마음 문제 해결 수 있도록 공감 행동지침 여 년 정신과 의사...
2    인생 마지막 더욱 간절해지는 선하고 아름다운 삶 향 고민 소산 고독 병 영원 사랑 ...
Name: intro, dtype: object

## 3) TF-IDF 

In [32]:
# 블용어 txt파일 리스트 가져오기
sw=pd.read_csv('./data/final/한국어불용어100.txt', sep = "\t",  engine='python', encoding = "utf-8",header=None)

In [33]:
stop_words=list(sw[0].dropna())

In [34]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(adult_final_df['intro'])

In [35]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [36]:
titles = adult_final_df['bookname']
indices = pd.Series(adult_final_df.index, index=adult_final_df['bookname'])

In [37]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [38]:
# ex1
get_recommendations('나는 죽을 때까지 재미있게 살고 싶다').head(10)

702    나이든 나와 살아가는 법 :흔들리지 않고 의연하게 나이 들 수 있는 후반생의 마음 사전 
427                           인생 수업 :잘 물든 단풍은 봄꽃보다 아름답다 
314              나는 내 나이가 참 좋다 :우아하고 지혜롭게 세월의 강을 항해하는 법 
289                      (어차피 살 거라면) 백 살까지 유쾌하게 나이 드는 법 
53                            하마터면 열심히 살 뻔했다 :야매 득도 에세이 
339                                   어른답게 삽시다 :이시형 에세이 
220                           백년을 살아보니 :인생의 황금기는 60~75세 
50     50부터는 인생관을 바꿔야 산다 :이제 자존심, 꿈, 사람은 버리고 오직 나를 위해...
2                          남아 있는 시간을 위하여:100세 철학자의 대표산문선
504          50 이후, 인생을 결정하는 열 가지 힘 - 하버드 심리학 거장의 마지막 강의
Name: bookname, dtype: object

In [40]:
# ex2
get_recommendations('아주 작은 습관의 힘 :최고의 변화는 어떻게 만들어지는가 ').head(10)

236        해빗 =내 안의 충동을 이겨내는 습관 설계의 법칙 /Habit 
312                         나는 습관을 조금 바꾸기로 했다 
858              (흔들리지 않고 끝까지 계속하게 만드는) 루틴의 힘 
528                습관의 힘 :반복되는 행동이 만드는 극적인 변화 
465        예술하는 습관 :위대한 창조의 순간을 만든 구체적 하루의 기록 
332               1만 시간의 재발견 :노력은 왜 우리를 배신하는가 
61                       12가지 인생의 법칙 :혼돈의 해독제 
188        미라클모닝 :당신의 하루를 바꾸는 기적 아침 6분이면 충분하다 
402               부자언니 부자연습 :가난한 공주 부자되기 프로젝트 
769    지금까지 산 것처럼 앞으로도 살 건가요? :내 인생의 판을 바꿀 질문 
Name: bookname, dtype: object

# B. 청소년들이 읽는 책을 책소개를 통해 비슷한 책 추천해주기

## 1) 도서관 데이터의 책 소개 크롤링으로 가져오기

- 데이터 불러오기

In [42]:
# excel 불러오기
teenager_book_revised=pd.read_excel('./data/final/content_re/teenager_book_revised.xlsx')

In [43]:
teenager_book_revised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 985 entries, 0 to 984
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                985 non-null    int64  
 1   ranking           985 non-null    int64  
 2   bookname          985 non-null    object 
 3   authors           985 non-null    object 
 4   publisher         985 non-null    object 
 5   publication_year  920 non-null    float64
 6   isbn13            985 non-null    int64  
 7   addition_symbol   920 non-null    float64
 8   vol               185 non-null    float64
 9   class_no          920 non-null    float64
 10  loan_count        985 non-null    int64  
 11  bookImageURL      984 non-null    object 
 12  keyword           546 non-null    object 
 13  describe          978 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 107.9+ KB


In [ ]:
# isbn 리스트 생성
teenager_isbn_list = list(teenager_book_revised['isbn13'].astype(str))

In [ ]:
# 자동화 웹 페이지 열기
path = './driver/chromedriver.exe'
driver = webdriver.Chrome(path)
driver

In [ ]:
isbn_url = 'http://www.kyobobook.co.kr/product/detailViewKor.laf?ejkGb=KOR&mallGb=KOR&barcode='
rest_url = '&orderClick=LAG&Kc='

In [ ]:
intro = []

for i in tqdm_notebook(range(len(teenager_isbn_list))) :
    url = isbn_url + str(teenager_isbn_list[i]) + rest_url
    driver.get(url)
    
    a_list = driver.find_elements_by_css_selector('div.box_detail_article')
    time.sleep(1.5)
    a_list = driver.find_elements_by_css_selector('div.box_detail_article')
    
    if len(a_list) > 0 :
        intro.append(a_list[0].text)
    else :
        intro.append('')
        continue

In [ ]:
teenager_book_revised['describe']=intro

In [ ]:
# 데이터 저장하기
teenager_book_revised.to_csv('./data/final/content_re/teenager_book_revised.csv',encoding='utf-8-sig',index=False)

In [44]:
# 제목 양쪽 공백제거
for i in range(len(teenager_book_revised)):
    teenager_book_revised['bookname'][i]=str(teenager_book_revised['bookname'][i]).strip()

In [46]:
# 책소개가 없는 데이터지우기
teenager_book_revised=teenager_book_revised[pd.notnull(teenager_book_revised['describe'])]

In [47]:
# 책 제목 중복 제거 
teenager_book_revised.drop_duplicates('bookname',keep='first',inplace=True)

In [48]:
# 인덱스 정리
teenager_book_revised.reset_index(drop=True,inplace=True)

In [49]:
teenager_book_revised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874 entries, 0 to 873
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                874 non-null    int64  
 1   ranking           874 non-null    int64  
 2   bookname          874 non-null    object 
 3   authors           874 non-null    object 
 4   publisher         874 non-null    object 
 5   publication_year  858 non-null    float64
 6   isbn13            874 non-null    int64  
 7   addition_symbol   813 non-null    float64
 8   vol               99 non-null     float64
 9   class_no          813 non-null    float64
 10  loan_count        874 non-null    int64  
 11  bookImageURL      874 non-null    object 
 12  keyword           534 non-null    object 
 13  describe          874 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 95.7+ KB


# C. 어린이들이 읽는 책을 책소개를 통해 비슷한 책 추천해주기

## ## 1) 도서관 데이터의 책 소개 크롤링으로 가져오기 - 앞의 작업과 반복작업이라 코드 생략

- 데이터 불러오기

In [4]:
child_final_df=pd.read_excel('./data/final/content_re/child_final_df.xlsx')

In [5]:
child_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                978 non-null    int64  
 1   ranking           978 non-null    int64  
 2   bookname          978 non-null    object 
 3   authors           978 non-null    object 
 4   publisher         978 non-null    object 
 5   publication_year  480 non-null    float64
 6   isbn13            978 non-null    int64  
 7   addition_symbol   932 non-null    float64
 8   vol               797 non-null    float64
 9   class_no          931 non-null    float64
 10  loan_count_x      978 non-null    int64  
 11  bookImageURL_x    977 non-null    object 
 12  keyword           512 non-null    object 
 13  describe          976 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 107.1+ KB


In [6]:
# 책소개가 없는 데이터지우기
child_final_df=child_final_df[pd.notnull(child_final_df['describe'])]

In [7]:
# 책 제목 중복 제거 
child_final_df.drop_duplicates('bookname',keep='first',inplace=True)

In [8]:
# 인덱스 정리
child_final_df.reset_index(drop=True,inplace=True)

In [9]:
# 제목 양쪽 공백제거
for i in range(len(child_final_df)):
    child_final_df['bookname'][i]=str(child_final_df['bookname'][i]).strip()

In [10]:
child_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                510 non-null    int64  
 1   ranking           510 non-null    int64  
 2   bookname          510 non-null    object 
 3   authors           510 non-null    object 
 4   publisher         510 non-null    object 
 5   publication_year  442 non-null    float64
 6   isbn13            510 non-null    int64  
 7   addition_symbol   464 non-null    float64
 8   vol               332 non-null    float64
 9   class_no          463 non-null    float64
 10  loan_count_x      510 non-null    int64  
 11  bookImageURL_x    510 non-null    object 
 12  keyword           439 non-null    object 
 13  describe          510 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 55.9+ KB


## 2) 책 소개를 형용사 + 명사 데이터로 바꾸기

In [11]:
book_intro=[]
for i in range(len(child_final_df)):
    sentences_tag = []
    lines=child_final_df['describe'][i].replace('\n',' ').split('.')
    twitter = Twitter()
    for sentence in lines:
        morph = twitter.pos(sentence)
        sentences_tag.append(morph)
    noun_adj_list = []
    for sentence1 in sentences_tag:
        for word, tag in sentence1:
            if tag in ['Noun','Adjective']:
                noun_adj_list.append(word)
    book_intro.append(noun_adj_list)

In [12]:
child_final_df['book_intro']=book_intro

In [13]:
intro=[]
for i in range(len(child_final_df)):
    s=' '.join(child_final_df['book_intro'][i])
    intro.append(s)

In [14]:
child_final_df['intro']=intro

In [15]:
child_final_df['describe'].head(3)

0    안전상식 학습만화 「쿠키런 서바이벌 대작전」 제4권 《산과 하늘 편》. 웨어울프맛 ...
1    추리로 배우는 교과서 과학\n\n『어린이 과학 형사대 CSI』시리즈 제5권《CSI,...
2    어린이용 그리스·로마 신화 베스트셀러. 아름다운 여신과 뛰어난 영웅, 그리고 신비로...
Name: describe, dtype: object

In [16]:
child_final_df['intro'].head(3)

0    안전 상식 학습만화 쿠키런 서바이벌 작전 제 권 산 하늘 편 웨어울프 맛 쿠키 도움...
1    추리 교과서 과학 어린이 과학 형사 시리즈 제 권 새로운 위기 처 이 시리즈 어린이...
2    어린이 용 그리스 로마 신화 베스트셀러 아름다운 여신 뛰어난 영웅 신비로운 일 만화...
Name: intro, dtype: object

In [ ]:
# 최종데이터 저장
child_final_df.to_csv('./data/final/content_re/child_final.csv',encoding='utf-8-sig',index=False)

## 3) TF-IDF 

In [17]:
sw=pd.read_csv('./data/final/한국어불용어100.txt', sep = "\t",  engine='python', encoding = "utf-8",header=None)

In [18]:
stop_words=list(sw[0].dropna())

In [20]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(child_final_df['intro'])

In [21]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [22]:
titles = child_final_df['bookname']
indices = pd.Series(child_final_df.index, index=child_final_df['bookname'])

In [24]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [25]:
# ex1
get_recommendations('쿠키런 서바이벌 대작전').head(10)

201                  쿠키런 서바이벌 대작전 21 (별똥별 산 편)
217        쿠키런 서바이벌 대작전 15 : 무인도 편 - 안전상식 학습만화
245          쿠키런 서바이벌 대작전 5 : 화재 편 - 안전상식 학습만화
251          쿠키런 서바이벌 대작전 6 : 동굴 편 - 안전상식 학습만화
156               쿠키런 서바이벌 대작전 26 : 사이보그의 역습 편
158                  쿠키런 서바이벌 대작전 23 : 차원의 문 편
184                쿠키런 서바이벌 대작전 24 : 쿠키볼의 비밀 편
198                 쿠키런 서바이벌 대작전 27 : 로봇의 심장 편
432    쿠키런 서바이벌 대작전 28 : 최후의 생존자 편 - 안전상식 학습만화
209        쿠키런 서바이벌 대작전 16 : 방사능 편 - 안전상식 학습만화
Name: bookname, dtype: object

In [26]:
# ex2
get_recommendations('나, 생일 바꿀래!').head(10)

3                                 이사도라 문
91                           마법의 설탕 두 조각
426    빈대 가족의 가난 탈출기 - 짠돌이들에게 배우는 경제 지혜!
38                       (기괴하고 요상한) 귀신딱지
88                           내 멋대로 아빠 뽑기
225                 빈대 가족의 덜렁이는 미운 우리 새끼
259                              할머니는 도둑
326                       엄마 사용법 :김성진 동화
353                            엄마가 사라진 날
246                          엄마는 게임 수업 중
Name: bookname, dtype: object